In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Download CSV") \
    .master("local[*]")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .getOrCreate()


24/07/04 19:51:55 WARN Utils: Your hostname, besgam-Aspire-E1-471 resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlp9s0)
24/07/04 19:51:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/besgam/.ivy2/cache
The jars for the packages stored in: /home/besgam/.ivy2/jars
:: loading settings :: url = jar:file:/home/besgam/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1fe27dac-7a69-4673-90b7-c88418ed1d31;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glass

In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import *

In [3]:
Schema = StructType() \
	.add('gestor', 'string') \
    .add('canalOrigem', 'string') \
    .add('regiao', 'string') \
    .add('uf', 'string') \
    .add('cidade', 'string') \
    .add('sexo', 'string') \
    .add('faixaEtaria', 'string') \
    .add('anoAbertura', 'integer') \
    .add('mesAbertura', 'integer') \
    .add('dataAbertura', 'string') \
    .add('dataResposta', 'string') \
    .add('dataAnalise', 'string') \
    .add('dataRecusa', 'string') \
    .add('dataFinalizacao', 'date') \
    .add('prazoResposta', 'string') \
    .add('prazoAnaliseGestor', 'integer') \
    .add('tempoResposta', 'integer') \
    .add('nomeFantasia', 'string') \
    .add('segmentoMercado', 'string') \
    .add('area', 'string') \
    .add('assunto', 'string') \
    .add('grupoProblema', 'string') \
    .add('problema', 'string') \
    .add('comoContratou', 'string') \
    .add('procurouEmpresa', 'string') \
    .add('respondida', 'string') \
    .add('situacao', 'string') \
    .add('avaliacaoReclamacao', 'string') \
    .add('notaConsumidor', 'integer') \
    .add('analiseRecusa', 'string')

In [4]:
inputDF = spark \
          .readStream \
          .option("maxFilesPerTrigger", 1) \
          .option("header", True) \
          .option("sep", ";") \
          .schema(Schema) \
          .csv("/home/besgam/consumidor/basecompleta*.csv") \
          .withColumn("datRefCarga", lit(substring(col('datafinalizacao'),1,7)))

In [5]:
query = inputDF.writeStream \
                .partitionBy('datRefCarga') \
                .format("delta") \
                .option("path", "hdfs://namenode:9000/data/consumidor/bronze") \
                .option("checkpointLocation", "hdfs://namenode:9000/tmp/checkpoint") \
                .start()

24/07/04 19:55:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/07/04 19:55:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Gestor, Canal de Origem, Região, UF, Cidade, Sexo, Faixa Etária, Ano Abertura, Mês Abertura, Data Abertura, Data Resposta, Data Análise, Data Recusa, Data Finalização, Prazo Resposta, Prazo Analise Gestor, Tempo Resposta, Nome Fantasia, Segmento de Mercado, Área, Assunto, Grupo Problema, Problema, Como Comprou Contratou, Procurou Empresa, Respondida, Situação, Avaliação Reclamação, Nota do Consumidor, Análise da Recusa
 Schema: gestor, canalOrigem, regiao, uf, cidade, sexo, faixaEtaria, anoAbertura, mesAbertura, dataAbertura, dataResposta, dataAnalise, dataRecusa, dataFinalizacao, prazoResposta, prazoAnaliseGestor, tempoResposta, nomeFantasia, segmentoMercado, area, assunto, grupoProblema, problema, comoContratou, pro

In [ ]:
query.awaitTermination()